In [10]:
import pandas as pd
import os
from pathlib import Path

In [11]:
def parse_log_file(log_file_path):
    """Parse the API cost log file and extract relevant information."""
    data = []
    
    with open(log_file_path, 'r') as f:
        current_entry = {}
        
        for line in f:
            line = line.strip()
            if not line:
                continue
                
            if line.startswith('['):
                if current_entry:
                    data.append(current_entry)
                current_entry = {'timestamp': line.strip('[]')}
            elif line.startswith('Game:'):
                current_entry['game'] = line.split(':')[1].strip()
            elif line.startswith('Model:'):
                current_entry['model'] = line.split(':')[1].strip()
            elif line.startswith('Total Input Tokens:'):
                current_entry['total_input_tokens'] = int(line.split(':')[1].strip())
            elif line.startswith('Input Text Tokens:'):
                current_entry['input_text_tokens'] = int(line.split(':')[1].strip())
            elif line.startswith('Input Image Tokens:'):
                current_entry['input_image_tokens'] = int(line.split(':')[1].strip())
            elif line.startswith('Output Tokens:'):
                current_entry['output_tokens'] = int(line.split(':')[1].strip())
            elif line.startswith('Total Input Cost:'):
                current_entry['input_cost'] = float(line.split('$')[1].strip())
            elif line.startswith('Total Output Cost:'):
                current_entry['output_cost'] = float(line.split('$')[1].strip())
            elif line.startswith('Total Cost:'):
                current_entry['total_cost'] = float(line.split('$')[1].strip())
                
        if current_entry:
            data.append(current_entry)
            
    return pd.DataFrame(data)


In [12]:

# List of log file paths - replace these with your actual paths
log_files = [
    "cache/ace_attorney/20250411_220206_Turnabout_Sisters_part1_vision-text_openai_o1-2024-12-17/ace_attorney_o1-2024-12-17_api_costs.log",
    # Add more log file paths here
]

# Process each log file
all_data = []
for log_file in log_files:
    try:
        df = parse_log_file(Path(log_file))
        df['log_file'] = log_file  # Add the log file path as a column
        all_data.append(df)
        print(f"Processed: {log_file}")
    except Exception as e:
        print(f"Error processing {log_file}: {str(e)}")

if not all_data:
    print("No valid data found in any log files")
else:
    # Combine all dataframes
    combined_df = pd.concat(all_data, ignore_index=True)
    
    # Convert timestamp to datetime
    combined_df['timestamp'] = pd.to_datetime(combined_df['timestamp'])
    
    # Sort by timestamp
    combined_df = combined_df.sort_values('timestamp')
    
    # Create a summary table of costs
    summary_df = pd.DataFrame({
        'Log File': combined_df['log_file'].unique(),
        'Total Input Cost': combined_df.groupby('log_file')['input_cost'].sum(),
        'Total Output Cost': combined_df.groupby('log_file')['output_cost'].sum(),
        'Total Cost': combined_df.groupby('log_file')['total_cost'].sum(),
        'Number of API Calls': combined_df.groupby('log_file').size()
    }).reset_index(drop=True)
    
    # Add a total row
    total_row = pd.DataFrame({
        'Log File': ['TOTAL'],
        'Total Input Cost': [summary_df['Total Input Cost'].sum()],
        'Total Output Cost': [summary_df['Total Output Cost'].sum()],
        'Total Cost': [summary_df['Total Cost'].sum()],
        'Number of API Calls': [summary_df['Number of API Calls'].sum()]
    })
    
    summary_df = pd.concat([summary_df, total_row], ignore_index=True)
    
    # Format the costs to 6 decimal places
    summary_df['Total Input Cost'] = summary_df['Total Input Cost'].map('${:.6f}'.format)
    summary_df['Total Output Cost'] = summary_df['Total Output Cost'].map('${:.6f}'.format)
    summary_df['Total Cost'] = summary_df['Total Cost'].map('${:.6f}'.format)
    
    print("\nCost Summary Table:")
    display(summary_df)
    
    # Save the summary to a CSV file
    summary_df.to_csv('api_costs_summary.csv', index=False)
    print("\nSummary saved to: api_costs_summary.csv")

Error processing cache/ace_attorney/20250411_220206_Turnabout_Sisters_part1_vision-text_openai_o1-2024-12-17/ace_attorney_claude-3-7-sonnet-20250219_api_costs.log: [Errno 2] No such file or directory: 'cache\\ace_attorney\\20250411_220206_Turnabout_Sisters_part1_vision-text_openai_o1-2024-12-17\\ace_attorney_claude-3-7-sonnet-20250219_api_costs.log'
No valid data found in any log files
